In [12]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

warnings.filterwarnings("ignore")

### Pre-processing

In [13]:
scotus_df = pd.read_csv("../Resources/SCOTUS_OHEncoded.csv", index_col=0)
df_109 = scotus_df[scotus_df["justice"] == 109]
df_109.tail()

,term_1953,term_1960,term_1962,term_1965,term_1968,term_1972,term_1973,term_1977,term_1984,term_1987,...,lawType_3,lawType_4,lawType_5,lawType_6,lawType_8,lawType_9,precedentAlteration_0,precedentAlteration_1,direction,justice
13803,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,109
13812,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,109
13820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,109
13829,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,2,109
13838,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,109


In [14]:
target_109 = df_109["direction"]
target_names = ["conservative", "liberal"]

In [15]:
data_109 = df_109.drop(["direction", "justice"], axis=1)
data_109.head()

,term_1953,term_1960,term_1962,term_1965,term_1968,term_1972,term_1973,term_1977,term_1984,term_1987,...,lawType_1,lawType_2,lawType_3,lawType_4,lawType_5,lawType_6,lawType_8,lawType_9,precedentAlteration_0,precedentAlteration_1
1264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1272,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1276,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [16]:
X = data_109
y = target_109
feature_names = data_109.columns

### Evaluating different classifiers

In [17]:
clf_lr = LogisticRegression()
clf_rf = RandomForestClassifier()
clf_nb = GaussianNB()
clf_svm = SVC(kernel="linear", probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf_lr, clf_rf, clf_nb, clf_svm], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

5-fold cross validation:

F1: 0.233 [Logistic Regression]
F1: 0.343 [Random Forest]
F1: 0.509 [Naive Bayes]
F1: 0.240 [Linear SVM]


In [18]:
# using all but logistic regression (lowest score) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[1], clf_rf),
                                            (labels[2], clf_nb),
                                            (labels[3], clf_svm)],
                                            voting = 'hard')

In [19]:
v_clf_soft = VotingClassifier(estimators = [(labels[1], clf_rf),
                                            (labels[2], clf_nb),
                                            (labels[3], clf_svm)],
                                            voting = 'soft')

In [20]:
new_labels = ["Random Forest", "Naive Bayes", "Linear SVM", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf_rf, clf_nb, clf_svm, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.302 [Random Forest]
F1: 0.509 [Naive Bayes]
F1: 0.240 [Linear SVM]
F1: 0.364 [Voting_Classifier_Hard]
F1: 0.495 [Voting_Classifier_Soft]


### Evaluating a trained Naive Bayes model

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [22]:
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [23]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.628
Test Acc: 0.555


In [24]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.45      0.31      0.37       170
     liberal       0.60      0.73      0.66       237

   micro avg       0.56      0.56      0.56       407
   macro avg       0.52      0.52      0.51       407
weighted avg       0.54      0.56      0.54       407



In [25]:
print(confusion_matrix(predictions, y_test))
predictions[:10], y_test[:10].ravel()

[[ 53  64]
 [117 173]]


(array([2, 1, 2, 2, 2, 1, 1, 1, 2, 2], dtype=int64),
 array([1, 2, 2, 2, 2, 1, 1, 2, 1, 2], dtype=int64))

### SVM Model Training

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [27]:
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [28]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.622
Test Acc: 0.604


In [29]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.58      0.19      0.28       170
     liberal       0.61      0.90      0.73       237

   micro avg       0.60      0.60      0.60       407
   macro avg       0.59      0.55      0.51       407
weighted avg       0.60      0.60      0.54       407



In [30]:
print(confusion_matrix(predictions, y_test))
predictions[:15], y_test[:15].ravel()

[[ 32  23]
 [138 214]]


(array([2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int64))

### Save Model and Confirm Load

In [31]:
import pickle

filename = '109_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [32]:
#Load Model
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6044226044226044
